In [15]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from get_decision_variable_map import get_decision_variable_map
from get_case_outputs_all_models import get_case_outputs_all_models
from get_unique_resources_data import get_unique_resources_data
from get_printable_resource_names import get_printable_resource_names
import h5py

In [16]:
import os

current_dir = os.getcwd()
print(current_dir)

c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures


In [17]:
plots_path = os.path.join(current_dir, 'plots') + "/"
pdf_path = os.path.join(current_dir, 'pdf_tables') + "/"
latex_path = os.path.join(current_dir, 'latex') + "/"
csv_path = os.path.join(current_dir, 'csv') + "/"
jpg_path = os.path.join(current_dir, 'jpg') + "/"
veps_path = os.path.join(current_dir, 'veps') + "/"
lookahead_avg_prices_path = os.path.join(veps_path, 'lookahead_avg_prices') + "/"

In [18]:
lookahead_profits_path = os.path.join(veps_path, 'lookahead_profits') + "/"
if not os.path.exists(lookahead_profits_path):
    os.makedirs(lookahead_profits_path)

In [19]:
# modeling scaling ModelScalingFactor
ModelScalingFactor = 1000

cem_path = os.path.join(os.path.dirname(current_dir), 'GenX.jl', 'research_systems')
policies_path = os.path.join(os.path.dirname(current_dir), 'SPCM', 'research_systems')

date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [20]:
case_names = [    
              "Thermal_Base",
              "2_Hr_BESS", 
              "2_Hr_BESS_Fuelx2",
              "4_Hr_BESS",
              "4_Hr_BESS_Fuelx2",
              "4_Hr_BESS_Fuelx3",
              "4_Hr_BESS_Fuelx4",
              "6_Hr_BESS",
              "6_Hr_BESS_Fuelx2",
              "8_Hr_BESS",
              "8_Hr_BESS_Fuelx2",
              "10_Hr_BESS",
              "10_Hr_BESS_Fuelx2",
              ]

policy_types = [
                'pf',
                'dlac-p',
                'dlac-i',
                'slac',
]

# Create a DataFrame with decision_variable_names as the index
decision_variable_map = get_decision_variable_map()

In [21]:
unique_resources, cases_resources_capacities = get_unique_resources_data(case_names, policies_path)


adding resource: NG 2-on-1 Combined Cycle (F-Frame) from case: Thermal_Base
adding resource: NG Combustion Turbine (F-Frame) from case: Thermal_Base
adding resource: Land-Based Wind - Class 1 - Technology 1 from case: Thermal_Base
adding resource: Utility PV - Class 1 from case: Thermal_Base
adding resource: Utility-Scale Battery Storage - 2Hr from case: 2_Hr_BESS
adding resource: Utility-Scale Battery Storage - 4Hr from case: 4_Hr_BESS
adding resource: Utility-Scale Battery Storage - 6Hr from case: 6_Hr_BESS
adding resource: Utility-Scale Battery Storage - 8Hr from case: 8_Hr_BESS
adding resource: Utility-Scale Battery Storage - 10Hr from case: 10_Hr_BESS


In [22]:
print_unique_resources = get_printable_resource_names(unique_resources)

In [23]:
print_unique_resources

['NG CC', 'NG CT', 'Wind', 'Solar', 'BESS']

In [24]:
# create a dataframe
cases_policies_df = pd.DataFrame(index=case_names, columns=[p for p in policy_types if p != 'pf'])

In [25]:
# create a dictionary of keys of print unique resources with cases_policies_df as the element
res_cases_policies_dict = {res: pd.DataFrame(index=case_names, columns=[p for p in policy_types if p != 'pf']) for res in print_unique_resources}

In [26]:
res_cases_policies_dict

{'NG CC':                   dlac-p dlac-i slac
 Thermal_Base         NaN    NaN  NaN
 2_Hr_BESS            NaN    NaN  NaN
 2_Hr_BESS_Fuelx2     NaN    NaN  NaN
 4_Hr_BESS            NaN    NaN  NaN
 4_Hr_BESS_Fuelx2     NaN    NaN  NaN
 4_Hr_BESS_Fuelx3     NaN    NaN  NaN
 4_Hr_BESS_Fuelx4     NaN    NaN  NaN
 6_Hr_BESS            NaN    NaN  NaN
 6_Hr_BESS_Fuelx2     NaN    NaN  NaN
 8_Hr_BESS            NaN    NaN  NaN
 8_Hr_BESS_Fuelx2     NaN    NaN  NaN
 10_Hr_BESS           NaN    NaN  NaN
 10_Hr_BESS_Fuelx2    NaN    NaN  NaN,
 'NG CT':                   dlac-p dlac-i slac
 Thermal_Base         NaN    NaN  NaN
 2_Hr_BESS            NaN    NaN  NaN
 2_Hr_BESS_Fuelx2     NaN    NaN  NaN
 4_Hr_BESS            NaN    NaN  NaN
 4_Hr_BESS_Fuelx2     NaN    NaN  NaN
 4_Hr_BESS_Fuelx3     NaN    NaN  NaN
 4_Hr_BESS_Fuelx4     NaN    NaN  NaN
 6_Hr_BESS            NaN    NaN  NaN
 6_Hr_BESS_Fuelx2     NaN    NaN  NaN
 8_Hr_BESS            NaN    NaN  NaN
 8_Hr_BESS_Fuelx2     NaN    Na

In [27]:
### preliminary data processing
# create a dictionary mapping the index to DateTime 
start_date = datetime.datetime(2018, 1, 1)
index_to_datetime = {i-1: start_date + datetime.timedelta(hours=i-1) for i in range(1, 8687)}
# create a list of DateTime from start_date to the end of the year 
datetime_list = [start_date + datetime.timedelta(hours=i) for i in range(8687)]

In [28]:
for case_name in case_names:

    # load generator characteristics from resources folder
    thermal_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Thermal.csv')
    vre_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Vre.csv')
    storage_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Storage.csv')

    # combine all resources to dfGen
    dfGen = pd.concat([thermal_dfGen, vre_dfGen, storage_dfGen], ignore_index=True)
    annual_cost_str = ['Inv_Cost_per_MWyr', 'Inv_Cost_per_MWhyr', 'Fixed_OM_Cost_per_MWyr']
    dfGen_annual_per_mw_costs = dfGen[annual_cost_str].copy()

    dfGen_annual_costs_sum = dfGen_annual_per_mw_costs.sum(axis=1)

    gen_capacity_gw = dfGen['Existing_Cap_MW'] / ModelScalingFactor
    gen_capacity_mw = dfGen['Existing_Cap_MW']
    for model in policy_types:
        case_model = case_name + '_' + model

        if model == 'pf':
            # decision_prices = pf_prices
            # results_path = policies_path + '\\' + case_name + '\\' + 'results_' + model + '\\'
            # selected_path = results_path + 'prices_scen_array.h5'
            continue
        elif model == 'dlac-p':
            results_path = policies_path + '\\' + case_name + '\\' + 'results_' + model + '\\'
        elif model == 'dlac-i':
            results_path = policies_path + '\\' + case_name + '\\' + 'results_' + model + '\\'
        elif model == 'slac':
            results_path = policies_path + '\\' + case_name + '\\' + 'results_' + model + '\\'
        else:
            print('Model not recognized')

        # read in correlated prices data
        correlated_prices = pd.read_csv(lookahead_avg_prices_path  + f'average_prices_comparison_{case_model}.csv')
        decision_prices = correlated_prices['Decision Prices']
        lookahead_prices = correlated_prices['Noon Lookahead Prices']
        
        # get a list of indices of datetime_list that of Datetimes in correlated_prices['DateTime']
        indices_of_interest = [datetime_list.index(datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')) for dt in correlated_prices['DateTime']]

        # read in generator dispatches...
        gen_dispatch = pd.read_csv(results_path + 'unit_pgen.csv')
        
        # read in generator names
        gen_charac = pd.read_csv(results_path + 'NetRevenue.csv')
        gen_names = gen_charac['generators'].values
        print_gen_names = get_printable_resource_names(gen_names)

        # add generator names to dispatch data
        # gen_dispatch['Resources'] = gen_names
        # transpose for easier reading
        # gen_dispatch_t = gen_dispatch.transpose()
        # # replace column names with generator names
        # gen_dispatch_t.columns = gen_names
        # ad a column of Datetime to gen dispatch t
        # gen_dispatch_t['DateTime'] = [index_to_datetime[i] for i in range(0, 8686)]

        # read in battery charges
        bess_charge = pd.read_csv(results_path + 'unit_charge.csv')
        # add a column of Datetime to bess charge t
        # bess_charge_t['DateTime'] = [index_to_datetime[i] for i in range(0, 8686)]

        # find the indices where correlated_prices['DateTime'] is in datetime_list
        indices_of_interest = [datetime_list.index(datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')) for dt in correlated_prices['DateTime']]
        # get the subset of gen_dispatch_t and bess_charge_t that correspond to the indices of interest
        gen_dispatch_t_subset = gen_dispatch.iloc[indices_of_interest]
        bess_charge_t_subset = bess_charge.iloc[indices_of_interest]

        # calculate profits at each hour for each generator
        # Reset the index of gen_dispatch_t_subset to align with decision_prices and lookahead_prices
        gen_dispatch_t_subset = gen_dispatch_t_subset.reset_index(drop=True)
        bess_charge_t_subset = bess_charge_t_subset.reset_index(drop=True)

        # Calculate profits at each hour for each generator
        decision_revenues = gen_dispatch_t_subset.multiply(decision_prices.values, axis=0)
        lookahead_revenues = gen_dispatch_t_subset.multiply(lookahead_prices.values, axis=0)

        decision_charge_costs = bess_charge_t_subset.multiply(decision_prices.values, axis=0)
        lookahead_charge_costs = bess_charge_t_subset.multiply(lookahead_prices.values, axis=0)

        decision_profits = decision_revenues - decision_charge_costs
        lookahead_profits = lookahead_revenues - lookahead_charge_costs

        # total each generator profits
        decision_gen_profits = decision_profits.sum(axis=0)
        lookahead_gen_profits = lookahead_profits.sum(axis=0)

        gen_capacity_mw.index = gen_names

        # calculate unit profit
        rt_unit_profit = decision_gen_profits / gen_capacity_mw
        da_unit_profit = lookahead_gen_profits / gen_capacity_mw

        # reindex the profits to the generator names
        dfGen_annual_costs_sum.index = gen_names
        dfGen_annual_costs_sum.index = gen_names

        # calculate normalized profits wrt to the annual costs
        rt_pmr = rt_unit_profit / dfGen_annual_costs_sum
        da_pmr = da_unit_profit / dfGen_annual_costs_sum
        # calculate the difference between decision and lookahead profits
        
        # create a dataframe for profits
        profits_df = pd.DataFrame({
            'Generator': print_gen_names,
            'RT Profits': decision_gen_profits.values,
            'DA Profits': lookahead_gen_profits.values,
            'RT Unit Profit': rt_unit_profit.values,
            'DA Unit Profit': da_unit_profit.values,
            'RT PMR': rt_pmr.values,
            'DA PMR': da_pmr.values,
        })

        dec_look_profits_diff = decision_gen_profits - lookahead_gen_profits
        profits_df['Decision - Lookahead Profits'] = (dec_look_profits_diff.values / ModelScalingFactor**2).round(2)

        profits_df['RT - DA PMR'] = ((rt_pmr.values.astype(float) - da_pmr.values.astype(float)) * 100).round(2)

        # add dec_look_profitts_diff to res_cases_policies_dict at the index of case_name and column of model
        for res in print_gen_names:
            if res in profits_df['Generator'].values:
                res_cases_policies_dict[res].loc[case_name, model] = profits_df.loc[profits_df['Generator'] == res, 'RT - DA PMR'].values[0]
            else:
                res_cases_policies_dict[res].loc[case_name, model] = np.nan


        profits_df.set_index('Generator', inplace=True)

        # save each profit to a pdf file
        fig, ax = plt.subplots()
        profits_df.plot(kind='bar', ax=ax)
        ax.set_title(f'Decision Profits vs Lookahead Profits for {case_name} - {model}')
        ax.set_ylabel('Profits')
        plt.tight_layout()
        plt.savefig(lookahead_profits_path + f'{case_name}_{model}_profits.eps', format='eps')
        plt.close()

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\ks885\\Documents\\aa_research\\Modeling\\spcm_genx_experiment\\figures\\veps/lookahead_avg_prices/average_prices_comparison_10_Hr_BESS_dlac-i.csv'

In [ ]:
# Create a new dictionary with case names as keys and generator names as row indices
case_gen_dict = {
    case: pd.DataFrame(
        {res: res_cases_policies_dict[res].loc[case] for res in print_unique_resources}
    ).T
    for case in case_names
}

In [ ]:
case_gen_dict

{'Thermal_Base':       dlac-p dlac-i   slac
 NG CC -18.28  18.59 -63.34
 NG CT   -0.8  27.31 -16.17
 Wind   -6.29  -2.17 -32.45
 Solar  -3.35  -2.95 -37.95
 BESS     NaN    NaN    NaN,
 '2_Hr_BESS':       dlac-p dlac-i   slac
 NG CC  12.22  47.82  33.38
 NG CT  20.75  55.36   43.4
 Wind     3.5  11.51   0.16
 Solar  -3.19   1.47   0.24
 BESS  -20.78   18.8  32.95,
 '2_Hr_BESS_Fuelx2':       dlac-p dlac-i   slac
 NG CC -23.98   44.2  32.64
 NG CT -29.61  45.93  34.13
 Wind    3.07  17.24  14.41
 Solar  -4.22   6.28   2.92
 BESS  -10.84  11.68  10.94,
 '4_Hr_BESS':       dlac-p dlac-i   slac
 NG CC  16.14  54.08  44.11
 NG CT  22.16  62.79  51.76
 Wind    6.62  13.34   7.48
 Solar  -2.43    6.0   6.85
 BESS    1.64  32.87   37.1,
 '4_Hr_BESS_Fuelx2':       dlac-p dlac-i   slac
 NG CC  -20.7  44.88  45.08
 NG CT -27.18  45.39  45.79
 Wind    6.21  21.89  21.55
 Solar   1.48  12.72   8.85
 BESS  -19.99  21.84  27.21,
 '4_Hr_BESS_Fuelx3':       dlac-p dlac-i   slac
 NG CC  -0.52  62.17  33.

In [ ]:
# in csv path, make a Table4 folder
table4_csv_path = os.path.join(csv_path, 'Table4') + "/"
if not os.path.exists(table4_csv_path):
    os.makedirs(table4_csv_path)

In [ ]:
# loop through each print_unique_resource and print the dataframe in res_cases_policies_dict to a csv and latex file
for res in print_unique_resources:
    res_cases_policies_dict[res].to_csv(table4_csv_path + f'Table4_{res}_RT-DA_pmr.csv')
    with open(latex_path + f'Table4_{res}_RT-DA_pmr.tex', 'w') as f:
        f.write(res_cases_policies_dict[res].to_latex())

In [ ]:
def save_case_gen_dict_latex(case, case_gen_dict, latex_path):
    with open(latex_path + f'Table4_{case}_RT-DA_pmr.tex', 'w') as f:
        f.write('\\begin{table}[htbp]\n')
        f.write('\\centering\n')
        f.write('\\begin{tabular}{l' + 'c' * len(case_gen_dict[case].columns) + '}\n')
        f.write('\\toprule\n')
        f.write('\\hline\n')
        f.write(' & ' + ' & '.join(case_gen_dict[case].columns) + ' \\\\\n')
        f.write('\\hline\n')
        for index, row in case_gen_dict[case].iterrows():
            # remove nan values from row
            row = row.dropna()
            f.write('\\quad ' + f'{index} & ' + ' & '.join([f'{int(val)}\%' for val in row]) + ' \\\\\n')
        f.write('\\hline\n')
        f.write('\\bottomrule\n')
        f.write('\\end{tabular}\n')
        case_print = case.replace('_', ' ')
        f.write('\\caption{RT-DA PMR for ' + case_print + '}\n')
        f.write('\\label{tab:table4_' + case + '}\n')
        f.write('\\end{table}\n')


In [ ]:
def write_all_cases_to_latex(case_names, case_gen_dict, latex_path):
    with open(latex_path + 'Table4_all_cases_RT-DA_pmr.tex', 'w') as f:
        f.write('\\begin{table*}[htbp]\n')
        f.write('\\centering\n')
        f.write('\\begin{tabular}{l' + 'c' * len(case_gen_dict[case_names[0]].columns) + '}\n')
        f.write('\\toprule\n')
        f.write('Case Name & ' + ' & '.join(case_gen_dict[case_names[0]].columns) + ' \\\\\n')
        f.write('\\midrule\n')
        for case in case_names:
            case_print = case.replace('_', ' ')
            f.write(f'{case_print} \\\\\n')
            for index, row in case_gen_dict[case].iterrows():
                # remove nan values from row
                row = row.dropna()
                f.write('\\quad ' + f'{index} & ' + ' & '.join([f'{int(val)}\%' for val in row]) + ' \\\\\n')
            f.write('\\midrule\n')
        f.write('\\bottomrule\n')
        f.write('\\end{tabular}\n')
        f.write('\\caption{RT-DA PMR for all cases.}\n')
        f.write('\\label{tab:table4_all_cases}\n')
        f.write('\\end{table*}\n')

In [ ]:
for case in case_names:
    case_gen_dict[case].to_csv(table4_csv_path + f'Table4_{case}_RT-DA_pmr.csv')
    save_case_gen_dict_latex(case, case_gen_dict, latex_path)

In [ ]:
write_all_cases_to_latex(case_names, case_gen_dict, latex_path)

In [ ]:
all_case_gen_df = pd.concat(
    [df.assign(
        Case=case_name,
        Resource=df.index
    ) for case_name, df in case_gen_dict.items()],
    ignore_index=True
)
# reorder such that Resource is the first column
all_case_gen_df = all_case_gen_df[['Resource'] + [col for col in all_case_gen_df.columns if col != 'Resource']]
all_case_gen_df.to_csv(table4_csv_path + 'Table4_all_cases_RT-DA_pmr.csv', index=False)
with open(latex_path + 'Table4_all_cases_RT-DA_pmr.tex', 'w') as f:
    f.write(all_case_gen_df.to_latex(index=False))